In [ ]:
# from __future__ import annotations


# # import sys
# # # caution: path[0] is reserved for script path (or '' in REPL)
# import sys
# import attr
# import torch
# import numpy as np
# from typing import Any
# from collections import defaultdict
# from pydantic import BaseModel, Extra
# from more_itertools import chunked
# from datasets import Dataset
# from bert_score.utils import get_tokenizer, get_model, model2layers
# from langchain.prompts.example_selector.base import BaseExampleSelector
# from prompts.base import ExampleTemplate
# from selector.base import CommonSelectorArgs, SelectorUtilsMixin
# from selector.greedy import decomposed_coverage_greedy
# from tools.track import track
# from datasets import load_dataset, Dataset
# from constants import ExSel as ES

from dataloader import create_dataloaders, load_noisy_dataset_by_task
from lora_model import LORAEngine
from influence import IFEngine

In [11]:
model_name_or_path="roberta-large"
task="mrpc"
noise_ratio=0.2
batch_size=32
target_modules=["value"]
device="cuda:4"
num_epochs=10
lr=3e-4

# print(name)

# mrpc_02_noise, noise_added=load_noisy_dataset_by_task(task="mrpc", noise_ratio=0.2)

# fine-tuning models
dataloader_outputs = create_dataloaders(model_name_or_path=model_name_or_path,
                                        task=task,
                                        noise_ratio=noise_ratio,
                                        batch_size=batch_size)
train_dataloader, eval_dataloader, noise_index, tokenized_datasets, collate_fn = dataloader_outputs

/home/haskari/miniconda3/envs/influence/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})
{'sentence1': ['Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .', "Yucaipa owned Dominick 's before selling the chain to Safeway in 1998 for $ 2.5 billion .", 'They had published an advertisement on the Internet on June 10 , offering the cargo for sale , he added .', 'Around 0335 GMT , Tab shares were up 19 cents , or 4.4 % , at A $ 4.56 , having earlier set a record high of A $ 4.57 .', 'The stock rose $ 2.11 , or about 11 percent , to close Friday at $ 21.51 on the New York Stock Exchange .', 'Revenue in the first quarter of the year dropped 15 percent from the same period a y

Saving the dataset (0/1 shards):   0%|          | 0/3668 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/408 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1725 [00:00<?, ? examples/s]

In [3]:
from datasets import load_from_disk,load_dataset

In [4]:
data=load_dataset('glue', 'mrpc')

data_train=load_from_disk('/nas02/Hadi/Incontenxt-influence/DataInf/datasets/mrpc-0.2-train.hf')
data_test=load_from_disk('/nas02/Hadi/Incontenxt-influence/DataInf/datasets/mrpc-0.2-test.hf')

data['train'] = data_train
data['validation'] = data_test

In [6]:
data['train'][0]

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
 'label': 0,
 'idx': 0}

In [8]:
print(noise_index[0:20])

[ 179 2679 3133 1061 1091 2230 2287  149  965  450   32 1807 1465  109
  415 3479  490  325  842  564]


In [ ]:
[ 179 2679 3133 1061 1091 2230 2287  149  965  450   32 1807 1465  109
  415 3479  490  325  842  564]

In [ ]:
Shot_ids: [[2420 2643 3066 ... 2550 1938 3643]
 [2420 3066 2643 ... 2550 1938 3643]
 [2420 2643 3066 ... 2550 1938 3643]
 ...
 [2420 3066 2643 ... 2550 1938 3643]
 [3596 2550 1938 ... 3554 3662 3643]
 [3066 2420 2643 ... 2550 1938 3643]]

In [ ]:
[[3509 2841 3660 ... 3643 3564 3066]
 [1456 3660  298 ... 3643 3564 3066]
 [3509 2561 3662 ... 3643 3564 3066]
 ...
 [2841 1456 3660 ... 3643 3564 3066]
 [3596 3509 3518 ... 3066 3564 3643]
 [3660  298 1456 ... 3643 3564 3066]]

In [12]:
dataloader_outputs = create_dataloaders(model_name_or_path=model_name_or_path,
                                        task=task,
                                        noise_ratio=noise_ratio,
                                        batch_size=batch_size)
train_dataloader, eval_dataloader, noise_index, tokenized_datasets, collate_fn = dataloader_outputs

print(noise_index[0:20])
lora_engine = LORAEngine(model_name_or_path=model_name_or_path,
                            target_modules=target_modules,
                            train_dataloader=train_dataloader,
                            eval_dataloader=eval_dataloader,
                            device=device,
                            num_epochs=num_epochs,
                            lr=lr,
                            low_rank=8, 
                            task=task)

lora_engine.build_LORA_model()
lora_engine.train_LORA_model()  

tr_grad_dict, val_grad_dict = lora_engine.compute_gradient(tokenized_datasets, collate_fn)

influence_engine = IFEngine()
influence_engine.preprocess_gradients(tr_grad_dict, val_grad_dict, noise_index)
influence_engine.compute_hvps(compute_accurate=False)
influence_engine.compute_IF()

# vals are identity , proposed , LiSSA 

# for method in influence_engine.IF_dict:



DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})
{'sentence1': ['Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .', "Yucaipa owned Dominick 's before selling the chain to Safeway in 1998 for $ 2.5 billion .", 'They had published an advertisement on the Internet on June 10 , offering the cargo for sale , he added .', 'Around 0335 GMT , Tab shares were up 19 cents , or 4.4 % , at A $ 4.56 , having earlier set a record high of A $ 4.57 .', 'The stock rose $ 2.11 , or about 11 percent , to close Friday at $ 21.51 on the New York Stock Exchange .', 'Revenue in the first quarter of the year dropped 15 percent from the same period a y

Saving the dataset (0/1 shards):   0%|          | 0/3668 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/408 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1725 [00:00<?, ? examples/s]

[ 179 2679 3133 1061 1091 2230 2287  149  965  450   32 1807 1465  109
  415 3479  490  325  842  564]


...

Epoch 9: {'accuracy': 0.696078431372549, 'f1': 0.8154761904761905}


100%|██████████| 13/13 [00:00<00:00, 14.48it/s]


Epoch 10: {'accuracy': 0.6985294117647058, 'f1': 0.8166915052160953}


100%|██████████| 408/408 [00:32<00:00, 12.55it/s]


NameError: name 'influence_version' is not defined

In [23]:
sorted(influence_engine.IF_dict['proposed'], reverse=True)

[746808.625,
 710301.75,
 543722.375,
 502382.5625,
 493554.71875,
 489649.59375,
 451517.78125,
 450479.375,
 434333.0625,
 431340.34375,
 413413.84375,
 396091.96875,
 377754.15625,
 368386.78125,
 365941.25,
 341499.15625,
 317318.40625,
 314836.75,
 309109.28125,
 305539.40625,
 299232.0,
 294177.125,
 291989.65625,
 289532.6875,
 287452.9375,
 278524.46875,
 268562.875,
 263903.65625,
 257664.125,
 241129.78125,
 238270.21875,
 236652.359375,
 231971.265625,
 226121.171875,
 222936.890625,
 217929.421875,
 208308.234375,
 208151.6875,
 207926.5625,
 203878.875,
 202862.09375,
 199996.328125,
 198116.875,
 194509.703125,
 190038.171875,
 188785.640625,
 184743.140625,
 183122.65625,
 182562.53125,
 180170.84375,
 179615.6875,
 179386.34375,
 174450.578125,
 163840.5625,
 161877.140625,
 160701.53125,
 159065.453125,
 154782.203125,
 153454.40625,
 152724.015625,
 152298.78125,
 152101.640625,
 151999.359375,
 151838.25,
 151154.421875,
 149422.921875,
 149246.546875,
 149221.078125

In [15]:
len(abs_vals)

3668

In [26]:
sorted(normalized_influence, reverse=True)

[1.0,
 0.9805666548003633,
 0.8918930968057464,
 0.8698870815880441,
 0.8651878424731594,
 0.8631090657448656,
 0.8428107335704376,
 0.8422579689990052,
 0.8336629614966181,
 0.8320698770007752,
 0.8225272398202089,
 0.8133064567107272,
 0.803544869598198,
 0.7985584271270348,
 0.7972566215636239,
 0.7842456025365055,
 0.7713737020537577,
 0.7700526664249046,
 0.7670038193970808,
 0.7651035029992913,
 0.7617459435541021,
 0.7590551317326023,
 0.7578906980380675,
 0.7565828040629335,
 0.7554757112296263,
 0.7507229073995828,
 0.7454201503139025,
 0.7429399542870071,
 0.7396185260720429,
 0.7308169617162001,
 0.7292947589754167,
 0.7284335398291918,
 0.7259416992990009,
 0.7228275765127041,
 0.721132519438003,
 0.7184669429186449,
 0.7133453909392456,
 0.7132620578831623,
 0.713142219309248,
 0.7109875536894777,
 0.7104463005413031,
 0.7089207957520852,
 0.707920324981452,
 0.7060001547055438,
 0.7036198685169228,
 0.7029531208932333,
 0.7008012166842443,
 0.6999386001976239,
 0.69964043

In [14]:
#abs_vals=[abs(x) for x in influence_engine.IF_dict['LiSSA']]
abs_vals=[x for x in influence_engine.IF_dict['proposed']]
max_val=max(abs_vals)
min_val=min(abs_vals)
normalized_influence=[]
for vals in abs_vals:
    #normalized_influence.append(1-((vals-min_val)/(max_val-min_val))) 
    normalized_influence.append((vals-min_val)/(max_val-min_val)) 

In [28]:
import numpy as np

In [77]:
argsorted_vals=np.argsort(influence_engine.IF_dict['LiSSA'])[::-1]

In [78]:
argsorted_vals

array([3066, 2264, 3643, ..., 1119, 3375, 2058])

In [79]:
count=0
for item in argsorted_vals[0:200]:
    if item in noise_index:
        count+=1

In [80]:
count

108